### Compare LPR models.

This script is to compare models that do direct end-to-end LPR against cropping vehicles first 

In [ ]:
import os
import json
import base64
import cv2
import numpy as np
from PIL import Image
from PIL import ImageDraw, ImageFont


from test_utils import make_local_request, draw_boxes_on_frame
import boto3

# Setup paths
IMAGE_DIR = "/home/gfuhr/projects/data/immediate_action/B8A44FB3A1F9__front_cars"

# AWS SageMaker config
SAGEMAKER_ENDPOINT = "sagemaker-inference-server-endpoint"
sagemaker_runtime = boto3.client("sagemaker-runtime")

# Colors
DETECTION_CLASS_COLORS = {
    "person": "green",
    "car": "orange",
    "truck": "purple",
    "motorcycle": "pink",
    "bus": "cyan",
    "bicycle": "yellow",
    "license_plate": "blue",
    "plate": "blue"
}

# Encode image to base64 for SageMaker
def encode_image_to_base64(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

# Make SageMaker inference call
def make_sagemaker_request(image_b64, model="license_plate_recognition"):
    response = sagemaker_runtime.invoke_endpoint(
        EndpointName=SAGEMAKER_ENDPOINT,
        ContentType="application/json",
        Body=json.dumps({
            "image_base64": image_b64,
            "model": model
        }),
    )
    return json.loads(response["Body"].read().decode())

# Optional: draw detections using your helper
def draw_all_detections(frame, detections):
    frame = draw_boxes_on_frame(
        frame,
        detections,
        label_fn=lambda d: f"{d['label']}: {d.get('score', 0):.2f}",
        color_fn=lambda d: DETECTION_CLASS_COLORS.get(d['label'], "red"),
        font_size=18
    )
    for det in detections:
        if 'license_plate' in det and isinstance(det['license_plate'], list):
            frame = draw_boxes_on_frame(
                frame,
                det['license_plate'],
                label_fn=lambda d: f"{d['ocr_text']} | {d['score']:.1f} | OCR: {d['ocr_confidence']:.1f}",
                color_fn=lambda d: DETECTION_CLASS_COLORS.get("plate", "blue"),
                font_size=18,
                label_position="bottom"
            )
    return frame


def draw_sagemaker_detections(frame, detections):
    draw = ImageDraw.Draw(frame)

    try:
        font = ImageFont.truetype("/usr/share/fonts/dejavu/DejaVuSansMono.ttf", 18)
    except:
        font = ImageFont.load_default()

    for det in detections:
        if det.get("bbox") is None or det.get("label") is None:
            continue

        label = det["label"]
        bbox = det["bbox"]
        x1 = bbox["top_left"]["x"]
        y1 = bbox["top_left"]["y"]
        x2 = bbox["bottom_right"]["x"]
        y2 = bbox["bottom_right"]["y"]

        if label == "license_plate":
            text = f'{det.get("ocr_text", "")} | {det.get("score", 0):.1f} | OCR: {det.get("ocr_confidence", 0):.1f}'
            color = "blue"
        else:
            text = f'{label}: {det.get("score", 0):.2f}'
            color = "red"

        draw.rectangle([x1, y1, x2, y2], outline=color, width=3)

        try:
            text_bbox = font.getbbox(text)
        except AttributeError:
            text_bbox = draw.textbbox((x1, y2), text, font=font)

        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        text_bg = [x1, y2, x1 + text_width + 4, y2 + text_height + 4]
        draw.rectangle(text_bg, fill=color)
        draw.text((x1 + 2, y2 + 2), text, fill="white", font=font)

    return frame

In [ ]:
output_dir = "results_concat"
os.makedirs(output_dir, exist_ok=True)

all_images = sorted([
    os.path.join(IMAGE_DIR, f)
    for f in os.listdir(IMAGE_DIR)
    if f.lower().endswith(".png")
])

for image_path in all_images:
    print(f"Processing: {os.path.basename(image_path)}")

    image_bgr = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    image_pil = Image.fromarray(image_rgb)

    local_result = make_local_request(
        image_path,
        ["object_detection_then_lpr"],
        classes_to_detect=["person", "car", "motorcycle", "bus", "truck", "bicycle"]
    )
    local_detections = (
        local_result.get("results", {})
        .get("object_detection_then_lpr", {})
        .get("detections", [])
    )

    image_b64 = encode_image_to_base64(image_path)
    sagemaker_result = make_sagemaker_request(image_b64)
    sagemaker_detections = sagemaker_result.get("detections", [])

    local_frame = draw_all_detections(image_pil.copy(), local_detections)
    sagemaker_frame = draw_sagemaker_detections(image_pil.copy(), sagemaker_detections)

    local_bgr = cv2.cvtColor(np.array(local_frame), cv2.COLOR_RGB2BGR)
    sagemaker_bgr = cv2.cvtColor(np.array(sagemaker_frame), cv2.COLOR_RGB2BGR)

    h = min(local_bgr.shape[0], sagemaker_bgr.shape[0])
    local_resized = cv2.resize(local_bgr, (int(local_bgr.shape[1] * h / local_bgr.shape[0]), h))
    sagemaker_resized = cv2.resize(sagemaker_bgr, (int(sagemaker_bgr.shape[1] * h / sagemaker_bgr.shape[0]), h))

    comparison = np.concatenate([local_resized, sagemaker_resized], axis=1)

    save_name = os.path.basename(image_path).replace(".png", "_compare.png")
    save_path = os.path.join(output_dir, save_name)
    cv2.imwrite(save_path, comparison)
